In [14]:
import os
import io
import sqlite3
import numpy as np

In [15]:
!ls ../data/numpy/P3B3/processed/

test_data.npy    test_labels.npy  train_data.npy   train_labels.npy


In [16]:
ARRY_PATH = '../data/numpy/P3B3/processed/'

In [17]:
def load_data(path, trainset=True):
    """ Load the P3B3 array data """
    if trainset:
        data = np.load(os.path.join(path, 'train_data.npy'))
        label = np.load(os.path.join(path, 'train_labels.npy'))
    else:
        data = np.load(os.path.join(path, 'test_data.npy'))
        label = np.load(os.path.join(path, 'test_labels.npy'))
        
    return (data, label)

In [18]:
def batch_arrays(x_train, y_train):
    """ Create a list of tuples of indexing into arrays """
    assert x_train.shape[0] == y_train.shape[0]
    return [(x_train[i], y_train[i]) for i in range(x_train.shape[0])]

In [19]:
x_train, y_train = load_data(ARRY_PATH)
x_test, y_test = load_data(ARRY_PATH, False)

array_data = batch_arrays(x_train, y_train)

In [20]:
def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

In [21]:
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [22]:
# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

In [25]:
conn = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()

In [26]:
with conn:
    cur.execute("""
        CREATE TABLE trainset (
            data array,
            label array
        )"""
    )

In [27]:
with conn:
    cur.executemany("INSERT INTO trainset VALUES (?,?)", array_data)

In [28]:
with conn:
    cur.execute("SELECT * from trainset")
    
data, label = cur.fetchone()

In [29]:
data

array([ 35, 197, 232, ...,   0,   0,   0])

In [30]:
label

array([5, 0, 1, 2])